In [1]:
import gym
import numpy as np
import pandas as pd
import random
from collections import defaultdict

In [2]:
env=gym.make("Blackjack-v0")

In [3]:
Q=defaultdict(float)

In [4]:
total_return=defaultdict(float)

In [5]:
N=defaultdict(int)

In [6]:
def epsioln_greedy_policy(state,Q):
#Sample a random value from the uniform distribution; 
#if the sampled value is less than epsilon then we select a random action, 
#else we select the best action that has the maximum Q value as shown. 
    epsilon=0.5
    if random.uniform(0,1)<epsilon:
        return env.action_space.sample()
    else:
        return max(list(range(env.action_space.n)),key=lambda x : Q[(state,x)])

In [7]:
num_timesteps=100

In [8]:
#Generate Episode
def generate_episode(Q):
    episode=[]
    state=env.reset()
    for t in range(num_timesteps):
        action=epsioln_greedy_policy(state,Q)
        next_state,reward,done,info= env.step(action)
        episode.append((state,action,reward))
        if done:
            break
        else:
            state=next_state
    return episode        

In [12]:
#Computing Optimal Policy
num_iterations=50000
for i in range(num_iterations):
    episode=generate_episode(Q)
    all_state_action_pairs=[(s,a) for (s,a,r) in episode]
    rewards=[r for (s,a,r) in episode]
    
    for t,(state,action,_) in enumerate (episode):
        if not (state,action) in all_state_action_pairs[0:t]:
            R=sum(rewards[t:])
            total_return[(state,action)]=total_return[(state,action)]+R
            N[(state,action)]+=1
            Q[(state,action)]=total_return[(state,action)]/N[(state,action)]
            
df=pd.DataFrame(Q.items(),columns=['state_action_pair','value'])

In [13]:
df.head(10)

,state_action_pair,value
0,"((21, 10, True), 0)",0.893487
1,"((21, 10, True), 1)",-0.260563
2,"((11, 7, False), 0)",-0.709091
3,"((11, 7, False), 1)",0.041420
4,"((14, 9, False), 0)",-0.523810
5,"((14, 9, False), 1)",-0.459044
6,"((15, 10, False), 0)",-0.585131
7,"((15, 10, False), 1)",-0.671908
8,"((11, 1, False), 0)",-0.838384
9,"((11, 1, False), 1)",-0.337793


In [11]:
df

,state_action_pair,value
0,"((21, 10, True), 0)",0.884323
1,"((21, 10, True), 1)",-0.251232
2,"((11, 7, False), 0)",-0.708333
3,"((11, 7, False), 1)",-0.024390
4,"((14, 9, False), 0)",-0.542857
...,...,...
555,"((15, 7, True), 1)",-0.583333
556,"((4, 3, False), 1)",-0.333333
557,"((4, 6, False), 1)",-0.333333
558,"((4, 3, False), 0)",-0.058824


In [14]:
df[124:126]

,state_action_pair,value
124,"((14, 7, False), 1)",-0.545894
125,"((21, 4, True), 0)",0.865443


In [20]:
np.argmax(df[0:1]['value'])

0